In [ ]:
import pandas as pd
import os

# GF TODO: check if excel present, otherwise download it
# wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

excel_with_path  = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.xlsx'
pickle_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.pkl'
csv_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.csv'

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster\n')
    df = pd.read_pickle(pickle_with_path)
else:
    print('-> picke does not exist, go to excel, and create it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)

df['Transaction'] = df.Quantity * df.Price
df=df.rename(columns={"Customer ID": "Customer_ID"})

# df.to_csv(csv_with_path, encoding = 'utf-8')

# GF TODO: there are nan 
# GF TODO: there are fields which are 'empty char', e.g. in the customer_id

In [ ]:
df.head(5)

##  Are there users at high risk of churning by the end of 2011

- https://www.kdnuggets.com/2019/05/churn-prediction-machine-learning.html 
- https://towardsdatascience.com/hands-on-predict-customer-churn-5c2a42806266
- https://www.google.com/search?q=dwarf+on+the+shoulders+of+giants&rlz=1C5CHFA_enCH771CH771&sxsrf=ALeKk034vLLmAHn5V1c0QAON4DZuO1GqVA:1582207255390&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiRrayrpeDnAhXCxaYKHZQVCgcQ_AUoAXoECBIQAw&biw=1418&bih=915#imgrc=oj4GwW0aHYfVYM
- similar exercise resolved here https://towardsdatascience.com/modeling-customer-churn-for-an-e-commerce-business-with-python-874315e688bf
- the repo of the library to be used https://github.com/CamDavidsonPilon/lifetimes
- the reference paper in pdf http://brucehardie.com/papers/018/fader_et_al_mksc_05.pdf http://brucehardie.com/papers/bgnbd_2004-04-20.pdf and the journal which has all aspects of a respectable journals 
- GF TODO: once you've found which customers have churned, make a few hitory plots to prove that indeed they have churned, to show that I am not blindly trusing the package I've downloade



- cancellations: fraction of cancellation by nunber of transactions and by proportion of renenue
-                correlation to CHURNING ? Correlation to country OR type of good purchased ?


- NOT SURE IF USEFUL some stack-overflow like explanations about the Pareto model https://stats.stackexchange.com/questions/251506/is-it-possible-to-understand-pareto-nbd-model-conceptually

In [ ]:
# pip install --user lifetimes
print('pip install worked out fine')

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from lifetimes.utils import *
from lifetimes import BetaGeoFitter,GammaGammaFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix, plot_period_transactions, plot_cumulative_transactions,plot_incremental_transactions
from lifetimes.generate_data import beta_geometric_nbd_model
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions,plot_history_alive

In [ ]:
help(summary_data_from_transaction_data)

##   RFM variables and negative/cancellation transactions
- frequency, recency, T
- how do I deal with transactions, in producing the RFM models ?
- for now just exclude them: cancellations are likely to correlate to churning behaviour, however the frequency of positive transactions is what the pareto model uses and expects to predict churning probabilty
- ===> check if there's correlation between churning behaviour and frequency of cancellations ?  

## FOR the moment I exclude cancellations from the type of transactions to compute RFM variables

In [ ]:
columns_forRFM = ['Customer_ID','InvoiceDate','Transaction']
df_forRFM = df[columns_forRFM][df['Transaction']>0]
#df_forRFM = df[['Customer_ID','InvoiceDate','Transaction']]

In [ ]:
df_forRFM.head(3)

In [ ]:
RFM = summary_data_from_transaction_data(df_forRFM,'Customer_ID','InvoiceDate',monetary_value_col='Transaction',freq='D')

In [ ]:
RFM.head(5)

## RFM info


DEFINITIONS
- Recency: time between initial purchase and most recent (last) purchase
- Frequency: number of repeat purchases made by a customer (total purchases — 1)
- Monetary: total spent on purchases

FUNCTION summary_data_from_transaction_data
- it's a rather clever thing: it AGGREGATES into 1 all transactions taking place at the same DAY!

In [ ]:
#RFM.describe()

# Verifiaction of how summary_data_from_transaction_data
- for a few Customer_Id randomly picked, Frequency makes sense 

# SHOULD I RESTRICT MOST OF THE ANALYSIS AND PLOTS TO CUSTOMERS WHO HAVE RETURNED?
- if a customer has haver come back, it's probably dead anyway...
- and all the values at 0 are somewhat disturbing

In [ ]:
# df_forRFM[df_forRFM['Customer_ID']==12346.0].sort_values(by=['InvoiceDate'],ascending=False)
#df_forRFM[df_forRFM['Customer_ID']==12346.0].InvoiceDate.unique()
# len(df[df['Customer_ID']==12346.0].InvoiceDate.unique())
df_forRFM[df_forRFM['Customer_ID']==12347.0].InvoiceDate.unique()

In [ ]:
RFM.columns

In [ ]:
tit='Num repeat purchases (tot - 1)'
RFM['frequency'].plot(kind='hist',logy=True,title=tit)

In [ ]:
tit='time bwn initial purchase and most recent (last) purchase'
RFM['recency'].plot(kind='hist',logy=True,title=tit)

In [ ]:
RFM.plot.scatter(x='frequency',y='recency')

In [ ]:
pd.plotting.scatter_matrix(RFM, alpha=0.2,figsize=(15,15))

# Assumptions of the BG/NBD model:
- A customer’s relationship has two phases: “alive” for an unobserved period of time, then “dead”
- While alive, the number of transactions made by a customer follows a Poisson distribution with transaction rate lambda
- Heterogeneity in lambda follows a gamma distribution
- After any transaction, a customer dies with probability p; the probability that a customer dies after a number of transactions follows a geometric distribution
- p follows a beta distribution
- Lambda and p vary independently across customers

# seen these references
- https://medium.com/data-shopify/how-shopify-merchants-can-measure-retention-c12284bfed6f
- http://brucehardie.com/papers/bgnbd_2004-04-20.pdf (of which I have other url's as well)


# For follow up

- how many customers are there with more than 50/75 purchases ?
  Perhaps worth selecting those out and treating them separately ?

In [ ]:
help(BetaGeoFitter)

In [ ]:
beta_gf = BetaGeoFitter(penalizer_coef=0.0)
beta_gf.fit(RFM['frequency'], RFM['recency'], RFM['T']);

In [ ]:
# https://github.com/CamDavidsonPilon/lifetimes/blob/master/lifetimes/plotting.py#L211
fig = plt.figure(figsize=(10,10))
plot_frequency_recency_matrix(beta_gf);

In [ ]:
fig = plt.figure(figsize=(10,10))
plot_probability_alive_matrix(beta_gf);

In [ ]:
fig = plt.figure(figsize=(10,10))
plot_period_transactions(beta_gf, max_frequency=250).set_yscale('log')

In [ ]:
fig = plt.figure(figsize=(10,10))
plot_period_transactions(beta_gf, max_frequency=250)

In [ ]:
data_star='2009-12-01'
calib_end='2011-05-31'
obser_end='2011-12-09'

from datetime import date
d_data_star=date( * map(lambda u:int(u), data_star.split('-') ) )
d_calib_end=date( * map(lambda u:int(u), calib_end.split('-') ) )
d_obser_end=date( * map(lambda u:int(u), obser_end.split('-') ) )

data_span = (d_obser_end - d_data_star).days
data_train= (d_calib_end - d_data_star).days
print('data cover overall %d days, of which %d are used for training'%(data_span,data_train))

In [ ]:
# https://github.com/CamDavidsonPilon/lifetimes/blob/master/lifetimes/utils.py#L27

#  Create a summary of each customer over a calibration and holdout period.
#  This function creates a summary of each customer over a calibration and holdout period
#  (training and testing, respectively).
    
summary_cal_holdout = calibration_and_holdout_data(df_forRFM,'Customer_ID','InvoiceDate',
                                                    calibration_period_end=calib_end,
                                                    observation_period_end=obser_end)

In [ ]:
summary_cal_holdout.sort_values(by=['frequency_cal'],ascending=False).head(5)

In [ ]:
fig = plt.figure(figsize=(10,10))

# non-store online retail between 01/12/2009 and 09/12/2011

beta_gf.fit(summary_cal_holdout['frequency_cal'], summary_cal_holdout['recency_cal'], summary_cal_holdout['T_cal'])
plot_cumulative_transactions(beta_gf, df_forRFM, 'InvoiceDate', 'Customer_ID', data_span, data_train);

In [ ]:
# is it correct we have only 25k transaction, as a result of aggregation of all of those into a single day?

In [ ]:
plot_incremental_transactions(beta_gf, df_forRFM,'InvoiceDate', 'Customer_ID', data_span, data_train);

In [ ]:
#k="frequency_cal"               # Purchases in calibration period,
#k="recency_cal"                 # Age of customer at last purchase,
#k="T_cal"                       # Age of customer at the end of calibration period,
#k="time_since_last_purchase"    # Time since user made last purchase

for k in ['frequency_cal','recency_cal','T_cal','time_since_last_purchase']:
    plot_calibration_purchases_vs_holdout_purchases(beta_gf, summary_cal_holdout,n=250,kind=k);

In [ ]:
# select only the customers who have come back already at least once
RFM_returned = RFM[RFM['frequency']>0]
RFM_returned['prob_alive'] = beta_gf.conditional_probability_alive(RFM_returned['frequency'],
                                                                   RFM_returned['recency'],RFM_returned['T'])
sns.distplot(RFM_returned['prob_alive'])

In [ ]:
tit='Alive probability'
RFM_returned['prob_alive'].plot(kind='hist',bins=20,title=tit)

In [ ]:
fig = plt.figure(figsize=(10,10))
sns.distplot(RFM_returned['prob_alive'], hist_kws={'log':True})

In [ ]:
RFM_returned['prob_alive'].plot(kind='hist',logy=True,bins=20,title=tit)

In [ ]:
sns.distplot(RFM_returned.loc[RFM_returned.prob_alive < 0.3]['prob_alive'])

In [ ]:
RFM_returned.columns

In [ ]:
prob_alive_sel = 0.1
RFM_returned.loc[RFM_returned.prob_alive < prob_alive_sel].plot.scatter(x='prob_alive',y='monetary_value')
# the customers at risk of churning are not very very valuable
print('there are %d customers with very low probability of being alive (i.e. %s or less)'% (len(RFM_returned.loc[RFM_returned.prob_alive < prob_alive_sel]),prob_alive_sel))

In [ ]:
pd.plotting.scatter_matrix(RFM_returned, alpha=0.2,figsize=(15,15))

## Have any users churned during the second half of 2011? 
- I define them as users who have a low prob of being alive at the end of 2011 and had higher prob mid 2011
## Are there any users at high risk of churning by the end of 2011?
- users who have low live probabiliy at the end of 2011 and didn't mid 2011 

- split the users in three populations: dead, alive, high rish
- answer to first  question: number of users who were alive|risk in may 2011 and are dead at the end of 2011
- answer to second question: number of users who are at high risk at the end of 2011
